In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/bgullen/Downloads/archive/data.csv')

/var/folders/d8/f00_49d1355_d53xn53c3dz00000gq/T/ipykernel_58480/2364901946.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,26,27,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/bgullen/Downloads/archive/data.csv')


In [3]:
df.head()

,OBJECTID,Shape,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,b'\x00\x01\xad\x10\x00\x00\xc8\xce\n[_@^\xc0\x...,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,1730.0,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County
1,2,b'\x00\x01\xad\x10\x00\x00\xc8\xe594\xe2\x19^\...,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1530.0,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County
2,3,b'\x00\x01\xad\x10\x00\x00x{\xac \x13/^\xc0@\x...,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,2024.0,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County
3,4,b'\x00\x01\xad\x10\x00\x00\xc8\x13u\xd7s\xfa]\...,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1400.0,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County
4,5,b'\x00\x01\xad\x10\x00\x00\xd0\x11y\xf8\xb6\xf...,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1200.0,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County


In [4]:
df.shape

(2303566, 39)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303566 entries, 0 to 2303565
Data columns (total 39 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   OBJECTID                       int64  
 1   Shape                          object 
 2   FOD_ID                         int64  
 3   FPA_ID                         object 
 4   SOURCE_SYSTEM_TYPE             object 
 5   SOURCE_SYSTEM                  object 
 6   NWCG_REPORTING_AGENCY          object 
 7   NWCG_REPORTING_UNIT_ID         object 
 8   NWCG_REPORTING_UNIT_NAME       object 
 9   SOURCE_REPORTING_UNIT          object 
 10  SOURCE_REPORTING_UNIT_NAME     object 
 11  LOCAL_FIRE_REPORT_ID           object 
 12  LOCAL_INCIDENT_ID              object 
 13  FIRE_CODE                      object 
 14  FIRE_NAME                      object 
 15  ICS_209_PLUS_INCIDENT_JOIN_ID  object 
 16  ICS_209_PLUS_COMPLEX_JOIN_ID   object 
 17  MTBS_ID                        object 
 18  MT

In [6]:
df_ca = df.loc[df['STATE'] == 'CA']

In [7]:
df_ca.shape

(251881, 39)

In [8]:
df_ca.head()

,OBJECTID,Shape,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,b'\x00\x01\xad\x10\x00\x00\xc8\xce\n[_@^\xc0\x...,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,1730.0,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County
1,2,b'\x00\x01\xad\x10\x00\x00\xc8\xe594\xe2\x19^\...,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1530.0,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County
2,3,b'\x00\x01\xad\x10\x00\x00x{\xac \x13/^\xc0@\x...,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,2024.0,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County
3,4,b'\x00\x01\xad\x10\x00\x00\xc8\x13u\xd7s\xfa]\...,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1400.0,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County
4,5,b'\x00\x01\xad\x10\x00\x00\xd0\x11y\xf8\xb6\xf...,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1200.0,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County


In [9]:
print(df_ca['FIRE_SIZE_CLASS'].value_counts())

FIRE_SIZE_CLASS
A    138489
B     96162
C     11802
D      2555
E      1389
F       946
G       538
Name: count, dtype: int64


In [10]:
df_ca.isnull().sum()

OBJECTID                              0
Shape                                 0
FOD_ID                                0
FPA_ID                                0
SOURCE_SYSTEM_TYPE                    0
SOURCE_SYSTEM                         0
NWCG_REPORTING_AGENCY                 0
NWCG_REPORTING_UNIT_ID                0
NWCG_REPORTING_UNIT_NAME              0
SOURCE_REPORTING_UNIT                 0
SOURCE_REPORTING_UNIT_NAME            0
LOCAL_FIRE_REPORT_ID             183302
LOCAL_INCIDENT_ID                 64596
FIRE_CODE                        187855
FIRE_NAME                         47369
ICS_209_PLUS_INCIDENT_JOIN_ID    248278
ICS_209_PLUS_COMPLEX_JOIN_ID     250760
MTBS_ID                          250434
MTBS_FIRE_NAME                   250434
COMPLEX_NAME                     250657
FIRE_YEAR                             0
DISCOVERY_DATE                        0
DISCOVERY_DOY                         0
DISCOVERY_TIME                    45065
NWCG_CAUSE_CLASSIFICATION             0


In [21]:
df_ca['FIPS_NAME'].unique()

array(['Plumas County', 'Placer County', 'El Dorado County',
       'Alpine County', 'Amador County', nan, 'Mono County',
       'Sierra County', 'Butte County', 'Yuba County', 'Orange County',
       'Nevada County', 'Trinity County', 'Del Norte County',
       'Los Angeles County', 'San Bernardino County', 'Fresno County',
       'San Diego County', 'Santa Barbara County', 'Mariposa County',
       'Riverside County', 'Monterey County', 'Shasta County',
       'Kern County', 'Siskiyou County', 'Tuolumne County',
       'Madera County', 'Lake County', 'Lassen County', 'Ventura County',
       'Humboldt County', 'Inyo County', 'Tehama County',
       'Calaveras County', 'Modoc County', 'Tulare County',
       'Colusa County', 'San Luis Obispo County', 'Mendocino County',
       'Glenn County', 'Stanislaus County', 'Imperial County',
       'Alameda County', 'San Benito County', 'Santa Clara County',
       'Sacramento County', 'Contra Costa County', 'Merced County',
       'Napa County

In [12]:
df['DISCOVERY_DATE'].head()

0     2/2/2005
1    5/12/2004
2    5/31/2004
3    6/28/2004
4    6/28/2004
Name: DISCOVERY_DATE, dtype: object